In [1]:
import sys
#for creating the mapper code
from sqlalchemy import Column, ForeignKey, Integer, String, Float

#for configuration and class code
from sqlalchemy.ext.declarative import declarative_base

#for creating foreign key relationship between the tables
from sqlalchemy.orm import relationship

#for configuration
from sqlalchemy import create_engine

#create declarative_base instance
Base = declarative_base()

#we'll add classes here

#creates a create_engine instance at the bottom of the file
engine = create_engine('sqlite:///transcriptomics.db')

# Base.metadata.create_all(engine)

In [2]:
import GEOparse
import re
import os
import pandas as pd
import numpy as np
from scipy import stats
from GSEpipeline import *

# find project root dir
currentdir = os.getcwd()
dirlist = currentdir.split('/')
projectdir = '/'.join(dirlist[0:-1])

In [3]:
class Sample(Base):
    __tablename__ = 'sample'

    id = Column(Integer, primary_key=True)
    ENTREZ_GENE_ID = Column(String(250), nullable=False)
    VALUE = Column(Float, nullable=False)
    P_VALUE = Column(Float)
    ABS_CALL = Column(String(2))
    Sample = Column(String(64))
    
    
class IDMapps(Base):
    __tablename__ = 'id_entrez_map'
    
    idx = Column(Integer, primary_key=True)
    ID = Column(String(250))
    ENTREZ = Column(String(250))

class GSEinfo(Base):
    __tablename__ = 'gseinfo'
    
    Sample = Column(String(64), primary_key=True)
    GSE = Column(String(64))
    Platform = Column(String(64))

In [4]:
Base.metadata.create_all(engine)

In [5]:
# create instance of project
gse2770 = GSEproject('GSE2770',projectdir)

14-Aug-2019 13:34:45 INFO GEOparse - Parsing ./GSE2770_family.soft.gz: 
14-Aug-2019 13:34:45 DEBUG GEOparse - DATABASE: GeoMiame
14-Aug-2019 13:34:45 DEBUG GEOparse - SERIES: GSE2770
14-Aug-2019 13:34:45 DEBUG GEOparse - PLATFORM: GPL96


Initialize project (GSE2770):
Root: /Users/zhzhao/Dropbox/Helikar/pipelines
Raw data: /Users/zhzhao/Dropbox/Helikar/pipelines/data/GSE2770_RAW


14-Aug-2019 13:34:47 DEBUG GEOparse - PLATFORM: GPL97
14-Aug-2019 13:34:48 DEBUG GEOparse - PLATFORM: GPL8300
14-Aug-2019 13:34:48 DEBUG GEOparse - SAMPLE: GSM60348
14-Aug-2019 13:34:48 DEBUG GEOparse - SAMPLE: GSM60349
14-Aug-2019 13:34:48 DEBUG GEOparse - SAMPLE: GSM60350
14-Aug-2019 13:34:48 DEBUG GEOparse - SAMPLE: GSM60351
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60352
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60353
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60354
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60355
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60356
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60357
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60358
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60359
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60360
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60361
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60362
14-Aug-2019 13:34:49 DEBUG GEOparse - SAMPLE: GSM60363
14-Aug-201

GPL96
[HG-U133A] Affymetrix Human Genome U133A Array
GPL97
[HG-U133B] Affymetrix Human Genome U133B Array
GPL8300
[HG_U95Av2] Affymetrix Human Genome U95 Version 2 Array


In [6]:
df_clean = gse2770.get_entrez_table_pipeline(fromcsv=True)
df_clean.sort_index(inplace=True)
df_clean

/usr/local/Cellar/ipython/7.5.0/libexec/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (101,104,107,110,113,116,119,122,125,128,131,134,137,140,143,146,149,152,155,158,161,164,167,170,173,176,179,182,185,188,191,194,197,200) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


,ENTREZ_GENE_ID,gsm60349.cel.gz,gsm60349.cel.gz.1,gsm60349.cel.gz.2,gsm60350.cel.gz,gsm60350.cel.gz.1,gsm60350.cel.gz.2,gsm60351.cel.gz,gsm60351.cel.gz.1,gsm60351.cel.gz.2,...,gsm60763.cel.gz.2,gsm60764.cel.gz,gsm60764.cel.gz.1,gsm60764.cel.gz.2,gsm60765.cel.gz,gsm60765.cel.gz.1,gsm60765.cel.gz.2,gsm60766.cel.gz,gsm60766.cel.gz.1,gsm60766.cel.gz.2
0,780 /// 100616237,717.599098,P,0.017000,966.038612,P,0.003067,724.167413,P,0.005643,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5982,140.340345,A,0.204022,124.764102,A,0.107301,90.986765,A,0.250724,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3310,68.989968,A,0.458816,55.205940,A,0.302547,97.283915,A,0.175989,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7849,1281.802247,P,0.017000,902.213482,P,0.019303,919.399037,P,0.027860,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2978,200.536466,A,0.218983,277.363337,A,0.060419,300.050853,A,0.117161,...,0.143002,373.110629,A,0.164032,626.187096,A,0.186972,401.599221,A,0.186972
5,7318 /// 100847079,540.953164,P,0.021866,255.370653,P,0.031336,506.635753,P,0.013092,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7067,223.734251,P,0.027860,158.550443,P,0.021866,189.557546,A,0.189687,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,11099,65.106109,A,0.541184,64.526428,A,0.602006,49.510810,A,0.602006,...,0.570568,90.043163,A,0.535424,114.817064,A,0.605162,89.731430,A,0.638946
8,6352,217.342576,P,0.009985,285.981770,P,0.000388,214.198185,P,0.035163,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1571,126.325873,P,0.039365,53.677210,A,0.418069,55.656157,A,0.267463,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Write to DATABASE

In [7]:
df_samples = pd.DataFrame([],columns=['ENTREZ_GENE_ID','VALUE','P_VALUE','ABS_CALL','Sample'])
df_samples.index.name = 'id'

for key,val in gse2770.gsm_platform.items():
    col_val = '{}.cel.gz'.format(key.lower())
    col_abs = '{}.cel.gz.1'.format(key.lower())
    col_p = '{}.cel.gz.2'.format(key.lower())
    if not col_val in list(df_clean):
        continue
    df_s = pd.DataFrame([])
    df_s.index.name='id'
    df_s['ENTREZ_GENE_ID'] = df_clean['ENTREZ_GENE_ID']
    df_s['VALUE'] = df_clean[col_val]
    df_s['ABS_CALL'] = df_clean[col_abs]
    df_s['P_VALUE'] = df_clean[col_p]
    df_s['Sample'] = key.upper()
    df_samples = pd.concat([df_samples, df_s.dropna(how='any')], ignore_index=True)

df_gseinfo = pd.DataFrame([],columns=['Sample','GSE','Platform'])
df_gseinfo['Sample'] = pd.Series(list(gse2770.gsm_platform.keys()))
df_gseinfo['Platform'] = pd.Series(list(gse2770.gsm_platform.values()))
df_gseinfo['GSE'] = gse2770.gsename
# df_database.dropna(how='any',inplace=True)
# df_database.reset_index(inplace=True)

/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [10]:
df_samples.to_sql(con=engine,name='sample',if_exists='replace',index_label='id')

In [14]:
df_gseinfo.to_sql(con=engine,name='gseinfo',if_exists='replace',index=False)

In [15]:
df = pd.read_sql(session.query(GSEinfo).filter().statement, session.bind)
df

,Sample,GSE,Platform
0,GSM60348,GSE2770,GPL8300
1,GSM60349,GSE2770,GPL8300
2,GSM60350,GSE2770,GPL8300
3,GSM60351,GSE2770,GPL8300
4,GSM60352,GSE2770,GPL8300
5,GSM60353,GSE2770,GPL8300
6,GSM60354,GSE2770,GPL8300
7,GSM60355,GSE2770,GPL8300
8,GSM60356,GSE2770,GPL8300
9,GSM60357,GSE2770,GPL8300


In [11]:
from sqlalchemy.orm import sessionmaker

DBSession = sessionmaker(bind=engine)

session = DBSession()

In [12]:
df = pd.read_sql(session.query(Sample).filter().statement, session.bind, index_col='id')
df

,ENTREZ_GENE_ID,VALUE,P_VALUE,ABS_CALL,Sample
id,,,,,
0,780 /// 100616237,717.599098,0.017000,P,GSM60349
1,5982,140.340345,0.204022,A,GSM60349
2,3310,68.989968,0.458816,A,GSM60349
3,7849,1281.802247,0.017000,P,GSM60349
4,2978,200.536466,0.218983,A,GSM60349
5,7318 /// 100847079,540.953164,0.021866,P,GSM60349
6,7067,223.734251,0.027860,P,GSM60349
7,11099,65.106109,0.541184,A,GSM60349
8,6352,217.342576,0.009985,P,GSM60349


In [32]:
aaa = session.query(Book).all()
aaa

 ...]

In [9]:
df_gseinfo

,Sample,GSE,Platform
0,GSM60348,GSE2770,GPL8300
1,GSM60349,GSE2770,GPL8300
2,GSM60350,GSE2770,GPL8300
3,GSM60351,GSE2770,GPL8300
4,GSM60352,GSE2770,GPL8300
5,GSM60353,GSE2770,GPL8300
6,GSM60354,GSE2770,GPL8300
7,GSM60355,GSE2770,GPL8300
8,GSM60356,GSE2770,GPL8300
9,GSM60357,GSE2770,GPL8300
